<a href="https://colab.research.google.com/github/sowji2604/ct_tasks/blob/main/RECOMMENDATION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [26]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tanujdhiman/post-recommendation-system")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/post-recommendation-system


In [27]:
# Load
df = pd.read_csv("/kaggle/input/post-recommendation-system/post_data.csv")
df

,user_id,post_id,time_stamp,Valuable
0,5eece14ffc13ae660900008b,136781766,01-01-2019 13:30,1
1,5eece14efc13ae660900003c,43094523,01-01-2019 13:33,2
2,5eece14efc13ae6609000025,42428071,01-01-2019 13:43,2
3,5eece14ffc13ae66090001d4,76472880,01-01-2019 13:54,2
4,5eece14ffc13ae66090000ac,202721843,01-01-2019 14:00,3
...,...,...,...,...
71795,5eece14ffc13ae660900018c,615389604,12/31/2019 12:37 AM,1
71796,5eece14ffc13ae660900010c,348689108,12/31/2019 12:50 PM,1
71797,5eece14ffc13ae6609000190,619052165,12/31/2019 12:51 AM,2
71798,5eece14efc13ae6609000067,426384418,12/31/2019 12:51 PM,1


In [28]:
df.columns = df.columns.str.strip()

In [29]:
if 'Valuable' in df.columns:
    print("The 'Valuable' column exists in the dataset.")
else:
    print("The 'Valuable' column does not exist in the dataset.")
if 'user_id' in df.columns:
    print("The 'user_id' column exists in the dataset.")
else:
    print("The 'user_id' column does not exist in the dataset.")
if 'post_id' in df.columns:
    print("The 'post_id' column exists in the dataset.")
else:
    print("The 'post_id' column does not exist in the dataset.")

The 'Valuable' column exists in the dataset.
The 'user_id' column exists in the dataset.
The 'post_id' column exists in the dataset.


In [30]:
# Encode IDs
user2idx = {id: i for i, id in enumerate(df['user_id'].unique())}
item2idx = {id: i for i, id in enumerate(df['post_id'].unique())}

df['user'] = df['user_id'].map(user2idx)
df['item'] = df['post_id'].map(item2idx)

# Train/test split
train, test = train_test_split(df[['user', 'item', 'Valuable']], test_size=0.2)

In [31]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim=50):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, latent_dim)
        self.item_emb = nn.Embedding(num_items, latent_dim)

    def forward(self, u, i):
        return (self.user_emb(u) * self.item_emb(i)).sum(1)

In [32]:
model = MF(len(user2idx), len(item2idx))
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

u = torch.tensor(train['user'].values)
i = torch.tensor(train['item'].values)
r = torch.tensor(train['Valuable'].values, dtype=torch.float)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    pred = model(u, i)
    loss = loss_fn(pred, r)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: loss = {loss.item():.4f}")

Epoch 1: loss = 60.3135
Epoch 2: loss = 57.3411
Epoch 3: loss = 54.4935
Epoch 4: loss = 51.7685
Epoch 5: loss = 49.1641
Epoch 6: loss = 46.6774
Epoch 7: loss = 44.3057
Epoch 8: loss = 42.0457
Epoch 9: loss = 39.8941
Epoch 10: loss = 37.8473
Epoch 11: loss = 35.9017
Epoch 12: loss = 34.0535
Epoch 13: loss = 32.2987
Epoch 14: loss = 30.6337
Epoch 15: loss = 29.0545
Epoch 16: loss = 27.5573
Epoch 17: loss = 26.1382
Epoch 18: loss = 24.7937
Epoch 19: loss = 23.5199
Epoch 20: loss = 22.3135
Epoch 21: loss = 21.1710
Epoch 22: loss = 20.0889
Epoch 23: loss = 19.0643
Epoch 24: loss = 18.0940
Epoch 25: loss = 17.1751
Epoch 26: loss = 16.3047
Epoch 27: loss = 15.4803
Epoch 28: loss = 14.6993
Epoch 29: loss = 13.9593
Epoch 30: loss = 13.2580
Epoch 31: loss = 12.5932
Epoch 32: loss = 11.9630
Epoch 33: loss = 11.3654
Epoch 34: loss = 10.7986
Epoch 35: loss = 10.2608
Epoch 36: loss = 9.7506
Epoch 37: loss = 9.2662
Epoch 38: loss = 8.8065
Epoch 39: loss = 8.3699
Epoch 40: loss = 7.9553
Epoch 41: loss

In [33]:
def recommend(user_id, top_k=5):
    model.eval()
    u_idx = torch.tensor([user2idx[user_id]] * len(item2idx))
    i_idx = torch.tensor(list(item2idx.values()))
    scores = model(u_idx, i_idx).detach().numpy()
    top_items = sorted(zip(item2idx.keys(), scores), key=lambda x: -x[1])[:top_k]
    return [item for item, _ in top_items]

# Example
print("Top picks:", recommend(user_id='5eece14efc13ae6609000067'))

Top picks: [np.int64(458195184), np.int64(602943065), np.int64(220861145), np.int64(495999294), np.int64(624861598)]
